In [ ]:
from flask import Flask, Response, render_template
import cv2
import numpy as np
import easyocr
import imutils

In [ ]:
# Initialize Flask app
app = Flask(__name__)

In [ ]:
# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

In [ ]:
# Function to process each frame
def process_frame(frame):
    try:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
        edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

        # Find contours
        keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(keypoints)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        # Loop over contours to find a rectangle
        location = None
        for contour in contours:
            approx = cv2.approxPolyDP(contour, 10, True)
            if len(approx) == 4:
                location = approx
                break

        if location is not None:
            mask = np.zeros(gray.shape, np.uint8)
            cv2.drawContours(mask, [location], 0, 255, -1)
            new_image = cv2.bitwise_and(frame, frame, mask=mask)
            (x, y) = np.where(mask == 255)
            (x1, y1) = (np.min(x), np.min(y))
            (x2, y2) = (np.max(x), np.max(y))
            cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

            # OCR on the cropped image
            result = reader.readtext(cropped_image)
            if result:
                text = result[0][-2]
                font = cv2.FONT_HERSHEY_SIMPLEX
                frame = cv2.putText(
                    frame,
                    text=text,
                    org=(approx[0][0][0], approx[1][0][1] + 60),
                    fontFace=font,
                    fontScale=1,
                    color=(0, 255, 0),
                    thickness=2,
                    lineType=cv2.LINE_AA
                )
                frame = cv2.rectangle(frame, tuple(approx[0][0]), tuple(approx[2][0]), (0, 255, 0), 3)

        return frame
    except Exception as e:
        print(f"Error processing frame: {e}")
        return frame


In [ ]:
# Video streaming generator
def generate_frames():
    camera = cv2.VideoCapture(0)  # Use the first webcam
    while True:
        success, frame = camera.read()
        if not success:
            break
        else:
            frame = process_frame(frame)
            _, buffer = cv2.imencode('.jpg', frame)
            frame_bytes = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')


In [ ]:
# Flask route to serve video feed
@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

# Flask route to serve the homepage
@app.route('/')
def index():
    return render_template('index.html')  # You can create a simple index.html

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True)